In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

ratings = pd.read_table('ml-1m/ratings.dat',  sep = '::', engine = 'python', header = None, names= ['UserID', 'MovieID', 'Rating', 'Timestamp'])
print(ratings.head())

   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [2]:
#read in movie data
movies = pd.read_table('ml-1m/movies.dat',  sep = '::', engine='python', encoding = "ISO-8859-1", header = None, names= ['ID', 'Title', 'Genre'])
print(movies.shape)

#rename id column in movie data so it matches the ratings data
movies.rename(columns = {'ID':'MovieID'}, inplace = True)

#merge the two on the movie id column
merged = pd.merge(ratings, movies, on = 'MovieID')


(3883, 3)


In [27]:
# merged['Genre'] = merged['Genre'].str.split('|')
# merged = merged.explode('Genre')
# print(merged)

# ratings = ratings.drop('Timestamp', axis=1)
# print(ratings)

In [34]:
full_ratings_matrix = merged.pivot(index='UserID', columns='MovieID', values='Rating')  # will i use that??

num_factors = 10
num_iter = 1
regularization = 0.05
learn_rate = 0.005


In [89]:
kf = KFold(n_splits = 5)
for trainid, testid in kf.split(ratings):
   
    users_matrix = pd.DataFrame(np.random.rand(len(np.unique(ratings['UserID'])), num_factors), index=np.unique(ratings['UserID']))    # users x features matrix
    movies_matrix = pd.DataFrame(np.random.uniform(low = 1.0, high = 5.0, size = (num_factors, len(np.unique(ratings['MovieID'])))), columns=np.unique(ratings['MovieID']))   # features x movies matrix
    ratings_matrix = pd.DataFrame(np.dot(users_matrix, movies_matrix), index=np.unique(ratings['UserID']), columns=np.unique(ratings['MovieID']))   # users x movies matrix

    #print(ratings_matrix)

    data_train = ratings.iloc[trainid]
    data_test = ratings.iloc[testid]

    iteration = 0
    while iteration != num_iter:
        for idx in range(len(data_train)):
            
            # print(ratings_matrix[data_train.iloc[idx]['UserID']])

            error = data_train.iloc[idx]['Rating'] - ratings_matrix[data_train.iloc[idx]['UserID']][data_train.iloc[idx]['MovieID']]

            users_matrix.iloc[data_train.iloc[idx]['UserID']] += learn_rate*(error*movies_matrix[data_train.iloc[idx]['MovieID']] - regularization*users_matrix.iloc[data_train.iloc[idx]['UserID']])
            movies_matrix[data_train.iloc[idx]['MovieID']] += learn_rate*(error*users_matrix.iloc[data_train.iloc[idx]['UserID']] - regularization*movies_matrix[data_train.iloc[idx]['MovieID']])
            ratings_matrix = pd.DataFrame(np.dot(users_matrix, movies_matrix), index=np.unique(ratings['UserID']), columns=np.unique(ratings['MovieID']))   # users x movies matrix

        iteration += 1



print(ratings_matrix)
    


1       11.678370
2       11.148249
3        7.813836
4        5.164946
5        7.599721
          ...    
6036    13.888710
6037    12.223284
6038    13.698476
6039     9.064834
6040     9.871851
Name: 1228, Length: 6040, dtype: float64
1       11.678370
2       11.148249
3        7.813836
4        5.164946
5        7.599721
          ...    
6036    13.888710
6037    12.223284
6038    13.698476
6039     9.064834
6040     9.871851
Name: 1228, Length: 6040, dtype: float64
1       11.678370
2       11.148249
3        7.813836
4        5.164946
5        7.599721
          ...    
6036    13.888710
6037    12.223284
6038    13.698476
6039     9.064834
6040     9.871851
Name: 1228, Length: 6040, dtype: float64
1       11.678370
2       11.148249
3        7.813836
4        5.164946
5        7.599721
          ...    
6036    13.888710
6037    12.223284
6038    13.698476
6039     9.064834
6040     9.871851
Name: 1228, Length: 6040, dtype: float64
1       11.678370
2       11.148249
3       

KeyError: 1229

In [19]:
# create matrix user_id x features (40)
# random values 0-1
users_matrix = np.random.rand(len(np.unique(data_train['UserID'])),40)

print(movies_matrix_train)

# create matrix features (40) x movies
# random values 1-5

[[2.02301432 3.66261155 4.60354012 ... 3.27811927 2.48930693 3.25545548]
 [1.18437018 1.1743911  1.49532274 ... 1.07488549 3.73386906 2.84122462]
 [3.7143788  1.87115321 3.32019844 ... 2.7697474  3.16581865 4.35680748]
 ...
 [3.35475869 1.63064013 4.33440768 ... 1.7604716  3.22567796 2.82499945]
 [1.03923755 2.35156941 2.4593082  ... 1.76095053 1.94044454 2.12394291]
 [1.73131323 4.44102838 3.54954886 ... 1.29323526 1.38812645 2.36479814]]
